# utils & framework

In [5]:
from collections.abc import Iterable

from IPython.core.display import Markdown
from sympy import *

tag_id = 0
var_id = 0

conditions = []
values = {}


def save_as_condition(condition):
    global tag_id, conditions
    cached_condition_id = cond_id
    conditions.append(condition)
    cond_id += 1
    return cached_condition_id


def substitute_values(expression):
    while True:
        left = False
        for (key, value) in values.items():
            before = str(expression)
            expression = expression.subs(key, value)
            expression = simplify(expression)
            if str(expression) != before:
                left = True

        if not left:
            break

    return expression


def save_as_variable(symbol, value):
    global values
    if symbol in values.keys():
        raise ValueError(f"variable {symbol} already exists")
    values[symbol] = substitute_values(value)


def get_conditions(ids) -> tuple:
    return tuple(map(lambda i: conditions[i], ids))


def beautiful_print(expression, message="", tag=None):
    # escape the message for LaTeX and append the expression
    inline = not message.endswith('$')
    tag_str = f'\\tag{{{tag}}}' if tag is not None else ""
    if inline:
        full_latex = message + '$' + latex(expression) + '$'
    else:
        full_latex = message + '$' + latex(expression) + tag_str + '$$'
    display(Markdown(full_latex))


def beautiful_peek(symbol, message="$",tag=None):
    value = substitute_values(symbol)
    beautiful_print(Eq(symbol, value, evaluate=False), message,tag)


def beautiful_solve(cond, variables):
    cond = [cond] if not isinstance(cond, Iterable) else cond
    variables = [variables] if not isinstance(variables, Iterable) else variables
    solution = solve(get_conditions(cond), tuple(variables), dict=True)[0]
    conclusions = []
    display(Markdown(f"${','.join(f"\\left ( {i} \\right ) " for i in cond)} \\Longrightarrow$"))
    for (variable, conclusion) in solution.items():
        beautiful_print(Eq(variable, conclusion), "$")
        save_as_variable(variable, conclusion)
        conclusions.append(conclusion)


def beautiful_condition(expression, message="$"):
    beautiful_print(expression(False) if callable(expression) else expression, message, tag_id)
    return save_as_condition(expression(True) if callable(expression) else expression)


def beautiful_compute(symbol, expression=None, message="$", tag: bool = False):
    beautiful_print(Eq(symbol, expression(False) if callable(expression) else expression, evaluate=False), message,
                    tag_id if tag else None)
    save_as_variable(symbol, expression)




# Define params

In [6]:
r = symbols('r')
t = symbols('t')
R = symbols('R')
epsilon = symbols('epsilon_0')
varphi = Function('varphi')(t)

theta = Function('theta')(t)
Rot_A = symbols('Rot_A')
Pos_A = symbols('Pos_A')


In [7]:
beautiful_compute(varphi, 0.5 * epsilon * t ** 2, "by const angular vel and zero init cond:$")
beautiful_compute(Pos_A, Matrix([(R + r) * cos(varphi), (R + r) * sin(varphi), 0]), "by geometry:$")
beautiful_compute(Rot_A,
                  Matrix([[cos(theta), -sin(theta), 0],
                [sin(theta), cos(theta), 0],
                [0, 0, 1]])
                  , "by geometry:$")


by const angular vel and zero init cond:$$\varphi{\left(t \right)} = 0.5 \epsilon_{0} t^{2}$$

by geometry:$$Pos_{A} = \left[\begin{matrix}\left(R + r\right) \cos{\left(\varphi{\left(t \right)} \right)}\\\left(R + r\right) \sin{\left(\varphi{\left(t \right)} \right)}\\0\end{matrix}\right]$$

by geometry:$$Rot_{A} = \left[\begin{matrix}\cos{\left(\theta{\left(t \right)} \right)} & - \sin{\left(\theta{\left(t \right)} \right)} & 0\\\sin{\left(\theta{\left(t \right)} \right)} & \cos{\left(\theta{\left(t \right)} \right)} & 0\\0 & 0 & 1\end{matrix}\right]$$

In [8]:
V_touch = symbols('V_touch')
cond_touch = beautiful_condition(Eq(V_touch, (R + r) * diff(varphi, t) - r * diff(theta, t)), "by geometry:$")
cond_rot = beautiful_condition(lambda e: Eq(V_touch, 0), "by pure roll on the large gear:$")
theta_d1 = diff(theta, t, 1)
beautiful_solve([cond_touch, cond_rot], [V_touch, theta_d1])


by geometry:$$V_{touch} = - r \frac{d}{d t} \theta{\left(t \right)} + \left(R + r\right) \frac{d}{d t} \varphi{\left(t \right)}\tag{0}$$

UnboundLocalError: cannot access local variable 'cond_id' where it is not associated with a value

In [11]:
solve1 = dsolve(Eq(theta_d1, substitute_values(theta_d1)), theta,
                # ics={theta.subs(t, 0): 0}
                )
beautiful_compute(theta, solve1.rhs, "solved:$")



solved:$$\theta{\left(t \right)} = C_{1} + \frac{R \epsilon_{0} t^{2}}{2 r} + \frac{\epsilon_{0} t^{2}}{2}$$

In [12]:
beautiful_peek(Rot_A, "by geometry:$",tag="answer")
beautiful_peek(Pos_A, "by geometry:$",tag="answer")

by geometry:$$Rot_{A} = \left[\begin{matrix}\cos{\left(C_{1} + \frac{R \epsilon_{0} t^{2}}{2 r} + \frac{\epsilon_{0} t^{2}}{2} \right)} & - \sin{\left(C_{1} + \frac{R \epsilon_{0} t^{2}}{2 r} + \frac{\epsilon_{0} t^{2}}{2} \right)} & 0\\\sin{\left(C_{1} + \frac{R \epsilon_{0} t^{2}}{2 r} + \frac{\epsilon_{0} t^{2}}{2} \right)} & \cos{\left(C_{1} + \frac{R \epsilon_{0} t^{2}}{2 r} + \frac{\epsilon_{0} t^{2}}{2} \right)} & 0\\0 & 0 & 1\end{matrix}\right]\tag{answer}$$

by geometry:$$Pos_{A} = \left[\begin{matrix}\left(R + r\right) \cos{\left(0.5 \epsilon_{0} t^{2} \right)}\\\left(R + r\right) \sin{\left(0.5 \epsilon_{0} t^{2} \right)}\\0\end{matrix}\right]\tag{answer}$$